In [1]:
import cv2
import torch
from ultralytics import YOLO

def detect_coke_bottle(image_path, model_path, real_height_cm=24):
    """
    Detects a  Coca-Cola bottle and calculates its real-world height.
    :param image_path: Path to the image.
    :param model_path: Path to the trained YOLO model.
    :param real_height_cm: Known real-world height of the  Coca-Cola bottle.
    :return: Bounding box height in pixels and estimated real-world height.
    """
    # Load the YOLO model
    model = YOLO(model_path)
    
    # Read the image
    image = cv2.imread(image_path)
    
    # Perform object detection
    results = model(image)
    
    # Get the detected bounding boxes
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0]  # Get bounding box coordinates
            height_pixels = y2 - y1  # Calculate height in pixels
            
            # Calculate Pixel-to-CM ratio
            pixel_to_cm_ratio = real_height_cm / height_pixels
            estimated_real_height = height_pixels * pixel_to_cm_ratio
            
            # print(f"Detected Coca-Cola Bottle Height: {height_pixels:.2f} pixels")
            # print(f"Estimated Real-World Height: {estimated_real_height:.2f} cm")
            
            return height_pixels, estimated_real_height
    
    print("No Coca-Cola bottle detected.")
    return None, None

# Example Usage
image_path = "Propellers test/IMG_3104.JPEG"  # Change to your image path
model_path = "best (8).pt"  # Path to your trained YOLO model
height_pixels, estimated_real_height =detect_coke_bottle(image_path, model_path)

print(f"Detected Coca-Cola Bottle Height: {height_pixels:.2f} pixels")
print(f"Estimated Real-World Height: {estimated_real_height:.2f} cm")


0: 800x608 1 750ml coke, 355.1ms
Speed: 4.2ms preprocess, 355.1ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 608)
Detected Coca-Cola Bottle Height: 238.16 pixels
Estimated Real-World Height: 24.00 cm


In [2]:
import torch
import cv2
import numpy as np
from ultralytics import YOLO

In [3]:
def load_model(model_path):
    model = YOLO(model_path)  # Load YOLO model
    return model

In [4]:
def detect_objects(model, image_path):
    image = cv2.imread(image_path)
    results = model(image)  # Perform object detection
    return results, image



In [5]:
def get_floor_coordinates(results):
    """
    Extracts the top Y-coordinate of the floor from Model (best(5).pt).
    """
    floor_y_top = None
    
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls.cpu().numpy().item())  # Get class ID
            y_min = int(box.xyxy[0][1])  # Top Y-coordinate
            
            if class_id == 1:  # Assuming class 1 is "floor"
                floor_y_top = y_min  # Top Y-coordinate of floor
    
    return floor_y_top

In [6]:
def get_propeller_coordinates(results):
    """
    Extracts the bottom Y-coordinates of multiple propellers from Model (best(9).pt).
    """
    propeller_y_bottoms = []  # List to store multiple propeller bottom Y-coordinates
    
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls.cpu().numpy().item())  # Get class ID
            y_max = int(box.xyxy[0][3])  # Bottom Y-coordinate
            
            if class_id == 0:  # Assuming class 0 is "propeller"
                propeller_y_bottoms.append(y_max)  # Store bottom Y-coordinate
    
    return propeller_y_bottoms

In [7]:
def calculate_height(propeller_y, floor_y, calibration_obj=None):
    """
    Calculates the real-world height from the floor to each propeller.
    Uses a calibration object if available.
    """
    if propeller_y is None or floor_y is None:
        print("Propeller or Floor not detected!")
        return None
    
    pixel_height = propeller_y - floor_y  # Distance in pixels
    
    if calibration_obj:
        real_height, obj_pixel_height = calibration_obj
        pixel_to_cm_ratio = real_height / obj_pixel_height
        real_propeller_height = pixel_height * pixel_to_cm_ratio
        return real_propeller_height
    
    return pixel_height  # Returns pixels if no calibration object

In [8]:
def display_propeller(image, propeller_boxes):
    """
    Displays the detected propeller(s) on the image.
    """
    for (x_min, y_min, x_max, y_max) in propeller_boxes:
        cropped_propeller = image[y_min:y_max, x_min:x_max]  # Crop detected propeller
        cv2.imshow("Detected Propeller", cropped_propeller)  # Show propeller
        cv2.waitKey(0)  # Wait for key press
        cv2.destroyAllWindows()  # Close window after key press

In [9]:
import os 
# Example usage
model_floor_path = "best (5).pt"  # Model detecting floor, ceiling, wall
model_propeller_path = "best (9).pt"  # Model detecting propellers
# image_path = "Propellers test/"  # Folder containing multiple images

# Calibration object (Coca-Cola bottle: 24 cm, detected height in pixels)
calibration_obj = (estimated_real_height,height_pixels)  # (Real height in cm, detected height in pixels)  height_pixels, estimated_real_height

# Load YOLO models
floor_model = load_model(model_floor_path)
propeller_model = load_model(model_propeller_path)


# Detect floor using model best(5).pt
floor_results, _ = detect_objects(floor_model, image_path)
floor_y_top = get_floor_coordinates(floor_results)

# Detect propellers using model best(9).pt
propeller_results, _ = detect_objects(propeller_model, image_path)
propeller_y_bottoms = get_propeller_coordinates(propeller_results)

if not propeller_y_bottoms or floor_y_top is None:
    print(f"No propellers or floor detected in {image_name}")
    exit()

print(f"\nImage: {image_path}")

    # Calculate heights for each detected propeller
for i, propeller_y_bottom in enumerate(propeller_y_bottoms):
    pixel_height = propeller_y_bottom - floor_y_top
    real_height = calculate_height(propeller_y_bottom, floor_y_top, calibration_obj)
        
    print(f"  Propeller {i+1}:")
    print(f"    - Pixel Height from Floor: {pixel_height} pixels")
    print(f"    - Real-World Height from Floor: {real_height:.2f} cm")

# propeller_boxes = get_propeller_coordinates(propeller_results)



'''if not propeller_y_bottoms or floor_y_top is None:
    print(f"No propellers or floor detected in {image_path}")
else:
    print(f"\nImage: {image_path}")
    display_propeller(image, propeller_boxes)  # Display detected propeller'''


0: 640x480 1 ceiling, 3 floors, 4 wallss, 328.7ms
Speed: 4.0ms preprocess, 328.7ms inference, 11.4ms postprocess per image at shape (1, 3, 640, 480)

0: 800x608 3 Propellers detection - v1 2025-02-20 11-43ams, 321.9ms
Speed: 4.5ms preprocess, 321.9ms inference, 0.7ms postprocess per image at shape (1, 3, 800, 608)

Image: Propellers test/IMG_3104.JPEG
  Propeller 1:
    - Pixel Height from Floor: -1699 pixels
    - Real-World Height from Floor: -171.21 cm
  Propeller 2:
    - Pixel Height from Floor: -1364 pixels
    - Real-World Height from Floor: -137.45 cm
  Propeller 3:
    - Pixel Height from Floor: -1316 pixels
    - Real-World Height from Floor: -132.62 cm


'if not propeller_y_bottoms or floor_y_top is None:\n    print(f"No propellers or floor detected in {image_path}")\nelse:\n    print(f"\nImage: {image_path}")\n    display_propeller(image, propeller_boxes)  # Display detected propeller'

In [ ]:
'''# Example usage
model_path = "best.pt"  # Path to trained YOLO model
image_folder = "Test_images/"  # Folder containing multiple images

# Calibration object (Coca-Cola bottle: 24 cm, detected height in pixels)
calibration_obj = (24, 120)  # (Real height in cm, detected height in pixels)

# Load YOLO model
model = load_model(model_path)

# Process multiple images
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)

    # Detect objects
    results, image = detect_objects(model, image_path)

    # Get multiple propeller bottom coordinates and floor top coordinate
    propeller_y_bottoms, floor_y_top = get_propeller_floor_coordinates(results)

    if not propeller_y_bottoms or floor_y_top is None:
        print(f"No propellers or floor detected in {image_name}")
        continue

    print(f"\nImage: {image_name}")
    
    # Calculate heights for each detected propeller
    for i, propeller_y_bottom in enumerate(propeller_y_bottoms):
        pixel_height = propeller_y_bottom - floor_y_top
        real_height = calculate_height(propeller_y_bottom, floor_y_top, calibration_obj)
        
        print(f"  Propeller {i+1}:")
        print(f"    - Pixel Height from Floor: {pixel_height} pixels")
        print(f"    - Real-World Height from Floor: {real_height:.2f} cm")'''